In [1]:
import pandas as pd
import time
import numpy as np
from heavy_tail_observations import BothSideWeibullNoise, BothSideParetoNoise, BothSideFrechetNoise
from heavy_tail_lin_bandit import MENU, TOFU, SupHvyLinBandit

from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
#import of review data
cols = ["user id","item id","rating","timestamp"]
#encoding using ISO-8859-1 is used because utf-8 does not support all the characters in movie names
df_data = pd.read_csv("ml-100k/u.data",sep="\t",names=cols,header=None,encoding="ISO-8859-1")
# df_data.head()

#import of moviedata
cols = ["movie id","movie title","release date","video release date","IMDb URL","unknown",
        "Action","Adventure","Animation","Childrens","Comedy","Crime","Documentary",
        "Drama","Fantasy","Film-Noir","Horror","Musical","Mystery","Romance","Sci-Fi",
        "Thriller","War","Western"]

df_movie = pd.read_csv("ml-100k/u.item",sep="|",names=cols,header=None,encoding="ISO-8859-1")
# df_movie.head()

#import of user data
cols = ["user id","age","gender","occupation","zip code"]
df_user = pd.read_csv("ml-100k/u.user",sep="|",names=cols,header=None,encoding="ISO-8859-1")
# df_user.head()

#frequency binning the ages into age groups as it will be easier for future analysis
df_user['age_group'] = pd.qcut(df_user['age'],q=10,precision=0)

#join all three dataframes

df = pd.merge(pd.merge(df_data,
                  df_user[["user id",
                           "age_group",
                           "gender",
                           "occupation"]],
                  on='user id',
                  how='left'),
              df_movie,
              left_on = 'item id',
              right_on = 'movie id',
              how ='left')
# df.head()

#drop unneccessary features
df.drop(["movie id",
        "movie title",
        "release date",
        "video release date",
        "IMDb URL",
        "unknown",
        "user id",
        "item id",
        "timestamp"],axis=1, inplace=True)
# df.head()

# check for null values
df.isnull().sum()

#categorize age_group, gender and occupation using 1-hot encoder
df['age_group'] = pd.Categorical(df['age_group'])
df['gender'] = pd.Categorical(df['gender'])
df['occupation'] = pd.Categorical(df['occupation'])

age_group_dummies = pd.get_dummies(df['age_group'])
gender_dummies = pd.get_dummies(df['gender'])
occupation_dummies = pd.get_dummies(df['occupation'])

df = pd.concat([df,
                age_group_dummies,
                gender_dummies,
                occupation_dummies], axis=1)

df.drop(['age_group',
        'gender',
        'occupation'], axis=1, inplace=True)
df.head()

,rating,Action,Adventure,Animation,Childrens,Comedy,Crime,Documentary,Drama,Fantasy,...,marketing,none,other,programmer,retired,salesman,scientist,student,technician,writer
0,3,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,3,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,1,0,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
y = df.rating.to_numpy()
X = df.drop('rating',axis=1).to_numpy()
dim = X.shape[1]
total_samples = len(y)

T = 10000
K = int(total_samples/T)

p=1.5
S = 1.

D = np.reshape(X, (T,K,dim))
R = np.reshape(y, (T,K))

In [12]:
get_mean = lambda x, y: R[y]
get_observation = lambda x, y, z: R[y][z]

In [13]:
error_list, theta_star = MENU(D, get_mean, get_observation, S=S, lamb=1., delta=0.01, p=p, c=1.)
# error_list, theta_star = TOFU(D, get_mean, get_observation, S=S, lamb=1., delta=0.01, p=p, b=1.)
# error_list, pro_estimator = SupHvyLinBandit(D, get_mean, get_observation, method="proof", S=S, lamb=1., delta=0.001, nu = 1e-4, p=p)
# error_list, btc_estimator = SupHvyLinBandit(D, get_mean, get_observation, method="btc", S=S, lamb=1., delta=0.001, nu = 1., p=p)
# error_list, bmm_estimator = SupHvyLinBandit(D, get_mean, get_observation, method="bmm", S=S, lamb=1., delta=0.001, nu = 1., p=p)

In [14]:
error_list

array([2.        , 2.        , 2.        , ..., 1.16129032, 1.16129032,
       1.16129032])